In [6]:
import datasets
import pandas as pd

dataset = datasets.load_dataset("trl-lib/tldr")

dataset['train'][0]

{'prompt': "SUBREDDIT: r/relationships\n\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\n\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My m

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


In [ ]:
print("Converting training dataset to tokens...")
def tokenize_chat(rows):
    chats = [[
        {"role": "system", "content": prompt},
        {"role": "assistant", "content": completion},
    ] for prompt, completion in zip(rows["prompt"], rows["completion"])]
    chat_text = [
        tokenizer.apply_chat_template(chat, tokenize=False)
        for chat in chats
    ]
    return tokenizer(chat_text, padding=True, truncation=True, max_length=1024, return_tensors="pt")

tokenized_dataset = dataset.map(tokenize_chat, batched=True)

Converting training dataset to tokens...


Map:   1%|          | 1000/116722 [00:00<00:55, 2099.47 examples/s]

Map: 100%|██████████| 6553/6553 [00:02<00:00, 2248.41 examples/s]


{'prompt': "SUBREDDIT: r/relationships\n\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\n\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My m

In [ ]:
from transformers import TrainingArguments, Trainer

output_dir = "Qwen2-0.5B-SFT"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    report_to = "wandb", 
    save_steps = 500,
    save_total_limit = 5,
    num_train_epochs=3,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)